In [2]:
# read data
import numpy as np
import ijson

from datetime import datetime
import time
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
import pandas as pd

import gensim
import gensim.downloader as api

# sys.getsizeof(tables)

In [3]:
singleline_json = "C:\\Users\\federico\\Documents\\MEGA\\UNI\\2-ML\\Progetto\\ijson\\singleline.json"
table_json = "C:\\Users\\federico\\Documents\\MEGA\\UNI\\2-ML\\Progetto\\results.json"
table_json_nb = "D:\\Progetto\\qatar150k2411.json"
table_json_results = "results.json"
table_json_nb_70k = "D:\\Progetto\\qatar70k1911.json"
table_json_theirs = "D:\\row_datsets\\theirs0401-1001.json"

In [10]:
def getAuthor2Author(conversation2tweets, tweet2author):
    author2author = []
    authors = set()
    for conversation in conversation2tweets:
        conversation_author = tweet2author.get(conversation)
        if conversation_author != None:
            for tweet in conversation2tweets.get(conversation):
                tweet_author = tweet2author.get(tweet)
                author2author.append((tweet_author, conversation_author))
                authors.add(conversation_author)
                authors.add(tweet_author)
    return authors, author2author



def dateToTimestampFromTwitter(date):
    dt = datetime.strptime(date, '%Y-%m-%dT%H:%M:%S.%fZ')
    return time.mktime(dt.timetuple())


def dateToTimestamp(date):
    dt = datetime.strptime(date, '%Y-%m-%d %H:%M:%S')
    return time.mktime(dt.timetuple())
       

In [11]:
dateToTimestampFromTwitter("2022-03-21T19:21:23.000Z")

1647886883.0

In [26]:

dataset = api.load("text8")
data = [d for d in dataset]

def tagged_document(list_of_list_of_words):
   for i, list_of_words in enumerate(list_of_list_of_words):
      yield gensim.models.doc2vec.TaggedDocument(list_of_words, [i])
data_training = list(tagged_document(data))

model = gensim.models.doc2vec.Doc2Vec(vector_size=768, min_count=2, epochs=30)

model.build_vocab(data_training)

In [2]:
modelGloveTwitter = api.load("glove-twitter-25")

In [6]:
dir(modelGloveTwitter)

['__class__',
 '__contains__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__len__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setitem__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_adapt_by_suffix',
 '_load_specials',
 '_log_evaluate_word_analogies',
 '_save_specials',
 '_smart_save',
 '_upconvert_old_d2vkv',
 '_upconvert_old_vocab',
 'add_lifecycle_event',
 'add_vector',
 'add_vectors',
 'allocate_vecattrs',
 'closer_than',
 'cosine_similarities',
 'distance',
 'distances',
 'doesnt_match',
 'evaluate_word_analogies',
 'evaluate_word_pairs',
 'expandos',
 'fill_norms',
 'get_index',
 'get_mean_vector',
 'get_normed_vectors',
 'get_vecattr',
 'get_vector',
 'has_index_for',
 'index2entity',
 'index2word',
 'index_to_key',
 'init_sims',
 

In [7]:
start = time.time()
vec = modelGloveTwitter.get_mean_vector("In 2005, Lionel Messi won the U20 World Cup with Argentina, picking up the Golden Boot and the Golden Ball. In 2022, he won the World Cup with Argentina, picking up the Golden Ball. 17 years is the longest gap between a player winning each tournament.")
print(vec)
print("time elapsed: " + str(time.time()-start))

[ 0.11750162  0.01895591 -0.04055125 -0.04210562 -0.01099056  0.01657312
  0.00866969  0.3377025  -0.00365594  0.01571392 -0.10260148 -0.17101242
 -0.674716    0.01287991 -0.16203086 -0.09484539 -0.16784452 -0.0077071
 -0.17401522  0.05319742 -0.03602245 -0.0722963   0.07011209  0.06712296
  0.05600847]
time elapsed: 0.0030050277709960938


In [28]:
model.train(data_training, total_examples=model.corpus_count, epochs=model.epochs)

In [29]:
def getBertGensim(text):
    return model.infer_vector(text)

In [45]:
start = time.time()
getBert("In 2005, Lionel Messi won the U20 World Cup with Argentina, picking up the Golden Boot and the Golden Ball. In 2022, he won the World Cup with Argentina, picking up the Golden Ball. 17 years is the longest gap between a player winning each tournament.")
print("time elapsed: " + str(time.time()-start))
start = time.time()
print(getBertGensim("In 2005, Lionel Messi won the U20 World Cup with Argentina, picking up the Golden Boot and the Golden Ball. In 2022, he won the World Cup with Argentina, picking up the Golden Ball. 17 years is the longest gap between a player winning each tournament.".split(" ")))
print("time elapsed: " + str(time.time()-start))

time elapsed: 0.2937498092651367
[ 0.30609906 -0.4257609  -0.00295583  0.6883677   0.5149538   0.41370568
 -0.39808396 -0.23946677 -0.11672528  0.2865635   0.25977862  0.12534496
  0.05975706 -0.30234292  0.00561945 -0.15289441  0.05387731  0.35352
 -0.35558093 -0.24215196 -0.27452496  0.05561659  0.0422972  -0.3473341
 -0.00371759 -0.8085954  -0.4056488  -0.32467645  0.1030391  -0.03923909
  0.04606949  0.38173416 -0.37970015  0.565544   -0.09377284 -0.14570056
 -0.1756006  -0.20266923  0.27907997  0.31263858 -0.42333114 -0.26852864
  0.27795598 -0.55369633  0.11871598  0.0556202   0.08023217  0.34378502
  0.41260037  0.0270313   0.04942469  0.28956997 -0.07367843 -0.3142051
 -0.21055073 -0.376073    0.25465533  0.28857037 -0.28357857  0.5202343
  0.24115601  0.39126354 -0.34072727  0.14941438  0.21069917  0.05546223
  0.36500275  0.20875432  0.18469909  0.00187685 -0.4128975  -0.2338749
 -0.69725525 -1.0556135  -0.2738392  -0.01350051 -0.18329176 -0.09395869
  0.17033212  0.37694278 

In [12]:
tweetCount = 0
df = pd.DataFrame(columns=["author_id", "tweet_id", "conversation_id", "timestamp", "text"])

with open(table_json_theirs, 'rb') as input_file:
    parser = ijson.parse(input_file, multiple_values=True)
    
    curr_author = 0
    curr_tweet = 0
    curr_text = ''
    curr_date = ''
    curr_conversation = 0
    
    curr_user_id = 0   
    curr_username = ''

    conversation2tweets = {}
    tweet2author = {}
    tweet2text = {}
    tweet2date = {}
    tweetIds = set()
    authors = set()
       
    for prefix, event, value in parser:
        if prefix=="includes.tweets.item.text":# or prefix=="data.item.text":
            if curr_text != '' and curr_tweet!=0:
                df.loc[len(df.index)] = [curr_author, curr_tweet, curr_conversation, curr_date, curr_text]
            curr_text = value
        if prefix=="includes.tweets.item.author_id":# or prefix=="data.item.author_id":
            curr_author = value
            authors.add(value)
        if prefix=="includes.tweets.item.id":# or prefix=="data.item.id":
            #tweetIds.add(value)
            tweetCount += 1
            curr_tweet = value
            tweet2author[value] = curr_author
            tweet2text[value] = curr_text
        if prefix=="includes.tweets.item.conversation_id":# or prefix=="data.item.conversation_id":
            #tweetIds.add(value)
            curr_conversation = value
            """if value != curr_tweet:
                if conversation2tweets.get(value) == None:
                    values = []
                    values.append(curr_tweet)
                    conversation2tweets[value] = values
                else:
                    values = conversation2tweets.get(value)
                    values.append(curr_tweet)"""
        if prefix=="includes.tweets.item.created_at":# or prefix=="data.item.created_at":
            curr_date = dateToTimestampFromTwitter(value)
            tweet2date[curr_tweet] = curr_date

df.to_json('df_theirs_nonormalize_withauthorid.json')

In [21]:
df_users = pd.DataFrame(columns=["user_id", "username"])
with open(table_json_nb, 'rb') as input_file:
    parser = ijson.parse(input_file, multiple_values=True)

    curr_id = 0   
    curr_username = ''
    
    for prefix, event, value in parser:
        if prefix=="data.item.entities.mentions.item.username":# or prefix=="data.item.text":
            if curr_id != 0 and curr_username !='':
                df_users.loc[len(df.index)] = [curr_id, curr_username]
            curr_username = value
        if prefix=="data.item.entities.mentions.item.id":# or prefix=="data.item.text":
            curr_id = value
        

df_users.to_json('df_authors_username_qatar.json')


In [33]:
df_users.sort_values(by='user_id')

,user_id,username
2696,1000314635361439744,pepele_news
115408,1000870679640670209,hgf_000
140624,1000870679640670209,hgf_000
88094,100101249,fotopine
87655,100101249,fotopine
...,...,...
19542,999256776213229570,NourNaim88
36624,999428734380859393,sarapeymanpour
148998,999428734380859393,sarapeymanpour
93486,999699017016619008,UmbroMex


In [3]:
df_users = pd.read_json("df_authors_username_qatar.json")
print(df_users[df_users['user_id']=='1153491723667984384'])

Empty DataFrame
Columns: [user_id, username]
Index: []


c:\Users\feder\anaconda3\envs\bert\lib\site-packages\pandas\core\computation\expressions.py:68: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  return op(a, b)


In [18]:
with open('list_tweetsauthorsids_qatar.txt', 'w') as f:
    for line in list(authors):
        f.write("%s\n"%line)

In [ ]:
df.to_json('df230k1124_gensim.json')

In [3]:
df = pd.read_json('dfqtr_final.json')

In [18]:
time.mktime(df['timestamp'][0].timetuple())

1669605303.0

In [30]:
df = pd.read_json('dfqtr_final.json')
df_tosave = pd.DataFrame(columns=["user_id", "item_id", "timestamp", "text"])
for index, row in df.iterrows():
    df_tosave.loc[len(df_tosave.index)] = [row['user_id'], row['item_id'], time.mktime(row['timestamp'].timetuple()), row['text']]

In [31]:
df_tosave.to_json('dfqtr_final_with_ts.json')

In [32]:
import csv
df_tosave = df_tosave.sort_values(by='timestamp')
df_tosave['timestamp'] = df_tosave['timestamp'] - df_tosave['timestamp'].min()

start = time.time()
with open('D:\dfqtr_final_with_ts.csv', 'w') as csvfile:
    filewriter = csv.writer(csvfile, delimiter=',',
                            quotechar='|', quoting=csv.QUOTE_MINIMAL)
    filewriter.writerow(['user_id','item_id','timestamp', 'state_label','comma_separated_list_of_features'])
    for index, row in df_tosave.iterrows():
        try:
            filewriter.writerow([row.user_id, row.item_id, row.timestamp, 0, *getBertGensim(row.text.split(" "))])
        except Exception as e:
            print(e)
end = time.time()
print("The time of execution of above program is :", (end-start) * 10**3, "ms")

The time of execution of above program is : 832521.0723876953 ms


In [33]:
from NormalizeTweets import normalizeTweet
start = time.time()
with open('D:\dfqtr_final_with_ts_norm.csv', 'w') as csvfile:
    filewriter = csv.writer(csvfile, delimiter=',',
                            quotechar='|', quoting=csv.QUOTE_MINIMAL)
    filewriter.writerow(['user_id','item_id','timestamp', 'state_label','comma_separated_list_of_features'])
    for index, row in df_tosave.iterrows():
        try:
            filewriter.writerow([row.user_id, row.item_id, row.timestamp, 0, *getBertGensim(normalizeTweet(row.text).split(" "))])
        except Exception as e:
            print(e)
end = time.time()
print("The time of execution of above program is :", (end-start) * 10**3, "ms")

The time of execution of above program is : 918865.5455112457 ms


In [34]:
df1 = pd.read_csv('D:\dfqtr_final_with_ts_norm.csv')
len(df1)

KeyboardInterrupt: 

In [ ]:
print(df['user_id'].value_counts().index.tolist()[:10])
print(df['user_id'].value_counts().values.tolist()[:10])

[240, 52, 1452, 1406, 38, 259, 55, 211, 241, 132]
[54, 45, 44, 42, 42, 39, 38, 35, 34, 33]


In [ ]:
print(df['item_id'].value_counts().index.tolist()[:10])
print(df['item_id'].value_counts().values.tolist()[:10])

[240, 1452, 52, 38, 1406, 259, 55, 211, 241, 132]
[56, 45, 45, 42, 42, 39, 37, 35, 34, 34]


## OLD

In [ ]:
tweetCount = 0
with open(table_json_nb, 'rb') as input_file:
    
    parser = ijson.parse(input_file, multiple_values=True)
    
    curr_author = 0
    curr_tweet = 0
    curr_text = ''

    conversation2tweets = {}
    tweet2author = {}
    tweet2text = {}
    tweet2date = {}
    tweetIds = set()
    authors = set()
       
    for prefix, event, value in parser:
        if prefix=="includes.tweets.item.text" or prefix=="data.item.text":
            curr_text = value
        if prefix=="includes.tweets.item.author_id" or prefix=="data.item.author_id":
            curr_author = value
            authors.add(value)
        if prefix=="includes.tweets.item.id" or prefix=="data.item.id":
            tweetIds.add(value)
            tweetCount += 1
            curr_tweet = value
            tweet2author[value] = curr_author
            tweet2text[value] = curr_text
        if prefix=="includes.tweets.item.conversation_id" or prefix=="data.item.conversation_id":
            tweetIds.add(value)
            if value != curr_tweet:
                if conversation2tweets.get(value) == None:
                    values = []
                    values.append(curr_tweet)
                    conversation2tweets[value] = values
                else:
                    values = conversation2tweets.get(value)
                    values.append(curr_tweet)
        if prefix=="includes.tweets.item.created_at" or prefix=="data.item.created_at":
            tweet2date[curr_tweet] = dateToTimestamp(value)
# vertices = set(tweet2author.values()) this returns all the authors, we are interested in that which had interactions
tweet2date = normalizeTimestamp(tweet2date)
vertices, edges = getAuthor2Author(conversation2tweets, tweet2author)
print("Tweets: " + str(tweetCount))
print("Vertices: " + str(len(vertices)))
print("Edges: " + str(len(edges)))

In [40]:
import csv
from NormalizeTweets import normalizeTweet
tweetIdsList = list(tweetIds)
authorsList = list(authors)

start = time.time()
with open('D:\qatar150k1124_gensim_norm.csv', 'w') as csvfile:
    filewriter = csv.writer(csvfile, delimiter=',',
                            quotechar='|', quoting=csv.QUOTE_MINIMAL)
    filewriter.writerow(['user_id','item_id', 'conversation_user_id','timestamp', 'state_label','comma_separated_list_of_features'])
    for conversation, tweets in zip(conversation2tweets, conversation2tweets.values()):
        for tweet in tweets:
            try:
                filewriter.writerow([authorsList.index(tweet2author[tweet]), tweetIdsList.index(conversation), authorsList.index(tweet2author[conversation]), tweet2date[tweet], 0, *getBertGensim(normalizeTweet(tweet2text[tweet]).split(" "))])
            except Exception as e:
                print(e)
end = time.time()
print("The time of execution of above program is :", (end-start) * 10**3, "ms")

    user_id item_id     timestamp  \
48       36      23  1.667571e+09   
127      41      88  1.668018e+09   
21       12      12  1.668040e+09   
99       14      28  1.668072e+09   
105      41      61  1.668075e+09   
116      26      49  1.668083e+09   
27       18      19  1.668091e+09   
93       36      60  1.668100e+09   
83        5      16  1.668101e+09   
72       17      28  1.668104e+09   
86        5       2  1.668113e+09   
34       20      25  1.668144e+09   
59       27       5  1.668144e+09   
61       27       5  1.668144e+09   
9         3       7  1.668144e+09   
81        4      39  1.668146e+09   
107      53      23  1.668150e+09   
29        1      21  1.668150e+09   
88       16      63  1.668151e+09   
76       26      16  1.668152e+09   

                                                  text  
48   \ ud83d \ udd 11 Key Player : SKOV OLSEN \ ud8...  
127           Karsdorp che cerca di crossare : HTTPURL  
21   Jose Mourinho : " Sassuolo kar \ u015f \ u0131

In [38]:

start = time.time()
with open('D:\\qatar150k1124_tweet2text.csv', 'w') as csvfile:
    filewriter = csv.writer(csvfile, delimiter=',',
                            quotechar='|', quoting=csv.QUOTE_MINIMAL)
    filewriter.writerow(['item_id','text'])
    for conversation, tweets in zip(conversation2tweets, conversation2tweets.values()):
        for tweet in tweets:
            try:
                filewriter.writerow([tweetIdsList.index(tweet), tweet2text[tweet]])
            except:
                print(tweet)
                print('no')
end = time.time()
print("The time of execution of above program is :", (end-start) * 10**3, "ms")

The time of execution of above program is : 2.058744430541992 ms


In [15]:

start = time.time()
with open('D:\\qatar15k1124_tweet2bert.csv', 'w') as csvfile:
    filewriter = csv.writer(csvfile, delimiter=',',
                            quotechar='|', quoting=csv.QUOTE_MINIMAL)
    filewriter.writerow(['item_id','comma_separated_list_of_features'])
    for conversation, tweets in zip(conversation2tweets, conversation2tweets.values()):
        for tweet in tweets:
            try:
                filewriter.writerow([tweetIdsList.index(tweet), *getBert(tweet2text[tweet]).numpy().tolist()])
            except:
                print(tweet)
                print('no')
end = time.time()
print("The time of execution of above program is :", (end-start) * 10**3, "ms")

1597555158623330304
no


In [7]:
#user_id,item_id,timestamp,state_label,comma_separated_list_of_features

import csv

tweetIdsList = list(tweetIds)
authorsList = list(authors)

start = time.time()
with open('D:\qatar40k1120.csv', 'w') as csvfile:
    filewriter = csv.writer(csvfile, delimiter=',',
                            quotechar='|', quoting=csv.QUOTE_MINIMAL)
    filewriter.writerow(['user_id','item_id', 'timestamp', 'state_label','comma_separated_list_of_features'])
    for conversation, tweets in zip(conversation2tweets, conversation2tweets.values()):
        for tweet in tweets:
            try:
                filewriter.writerow([authorsList.index(tweet2author[tweet]), tweetIdsList.index(conversation), tweet2date[tweet], 0, *getBert(tweet2text[tweet]).numpy().tolist()])
            except:
                print(tweet)
                print('no')
end = time.time()
print("The time of execution of above program is :", (end-start) * 10**3, "ms")

In [36]:
import csv
with open('D:\dfqtr_final_with_ts.csv') as input, open('tgn/data/dfqtr_final_with_ts.csv', 'w', newline='') as output:
    writer = csv.writer(output)
    for row in csv.reader(input):
        if any(field.strip() for field in row):
            writer.writerow(row)

user_id  \
220494 220494 429248729.0 0 -0.037920  0.033194  0.091983  0.122681 -0.018020 -0.203349 -0.139147  0.113392 -0.181800  0.027564  0.076301 -0.090996 -0.033413 -0.251577 -0.082217 -0.146891  0.108839  0.106048 -0.015899 -0.095562  0.040346 -0.026131  0.018893 -0.051091  0.197944  0.030847  0.007104 -0.084210  0.071762  0.017990  0.061542 -0.037267 -0.082913  0.012455 -0.049890  0.007076  0.133670 -0.035435 -0.199483 -0.034411  0.076047 -0.113197  0.122918 -0.060726  0.162931 -0.031118 -0.168706  0.004221  0.187681  0.012593  0.072056  0.091485 -0.032617  0.099886 -0.039738  0.052680  0.017087  0.018561 -0.133213  0.038122  0.006751 -0.025589  0.011257  0.060017 0.036851  0.040178 -0.029305 -0.002550 -0.054096 -0.286806 -0.227840 -0.000870 -0.076159 -0.057132 -0.004396  0.038742  0.022094 -0.052977  0.147470 -0.011518  0.075903 -0.044785 -0.000081 -0.016551 -0.154820 -0.007082  0.061539  0.015736  0.214928 -0.101854 -0.091245 -3.751632e-02  0.188270 -0.160483  0.062422  0.188304  0.033345  0.049271  0.117216  0.009159 -0.177715 -0.120913 -0.229223  0.058061 -0.043385 -0.007425 -0.149826  0.093426  0.052634  0.101701 -0.001871 -0.057622 -0.027922 -0.168993 -0.095618  0.141471 -0.076463  0.169676  0.060919  0.092261  0.045153  0.305541 -0.092427 -0.015505 -0.079958 -0.060797  0.063698 -0.013118  0.175971 -0.096640 -0.117354  0.063415 -0.083092 -0.080910  0.028310 -0.088986 -0.074522  0.135543 -0.067597  0.094754  0.027498  0.011142  0.061979  0.052696 -0.053851  0.121703  0.101069  0.116314  0.035354 -0.121382 -0.009230 -0.042883  0.198775 -0.201020  0.045801 0.073207  0.207921 -0.267248 -0.075775 -0.085950 -0.042308  0.236199  0.147400 -0.011789 -0.085248 -0.031545 -0.062680  0.063208 -0.032981  0.125550 -0.099315  0.040703  0.035241  0.131301  0.038218 0.224962  0.165276  0.086742  0.026877 -0.050868  0.093782  0.119621 -0.074580 -0.113641 -0.036277  0.069169 -0.063189 -0.145686 -0.025751 -0.004526 -0.143011 -0.077638 -0.151030 -0.037848 -0.016668  0.203784  0.107824  0.081158 -0.027474  0.008725 -0.062826 -0.119260  0.227142 -0.056765  0.049756  0.015326 -0.144948  0.095850  0.134023  0.064317 -0.199471 -0.045428  0.104384 -0.055369  0.040800 -0.047701 -0.018627 -0.079659  0.159095  0.039556 -0.091568 -0.029474  0.126297  0.039739 -0.059854  0.356851 -0.007361  0.042857 -0.076548  0.085189 -0.144028 -0.209315  0.100970 -0.228786  0.068820 -0.113679 -0.133792  0.224556  0.082150  0.060097 -0.001917  0.093897  0.027033  0.100991  0.013985  0.108526 -0.078654  0.054404 -0.019242  0.055676 -0.171847 -0.267451 -0.071987 -0.085274 -0.056728 -0.049420 -0.033637 -0.080820 -0.051141 -0.049348 0.116873  0.109893  0.139240 -0.060370 -0.219571 -0.034214  0.048649  0.060691 -0.094335  0.100101 -0.221332 0.048051  0.050802 -0.001380  0.208652  0.043213  0.038334 -0.198639 -0.164868 -0.086896 -0.067796  0.028455  0.000755 -0.036164 -0.025858  0.184134  0.052741  0.100359  0.013587 -0.102715 -0.060877 -0.095418 -0.178548  0.133553 -0.012378 -0.040684 -0.051936  0.032492  0.114218 -0.212967  0.095468 0.409896 -0.032557  0.142968 -0.046649 -0.009091  0.130306  0.016680  0.035393 -0.090255 -0.115457 -0.090926  0.246822  0.153889  0.132916 -0.114940  0.035698  0.061484  0.063136 -0.004296 -0.043692  0.256906 -0.002344  0.290841 -0.023142 -0.168452  0.118756  0.117865  0.100473  0.123098  0.012029  0.075107 -0.162386  0.030920  0.162566 -0.204907 -0.023728 -0.086566 -0.073324 -0.149558 -0.014767  0.112555  0.025683  0.111001 -0.186361 -0.065400  0.079295  0.221722 -0.058494  0.112868 -0.172364  0.224242 -0.062104 -0.009160 -0.092024  0.116256 -0.065459 -0.063038 -0.069005 -0.064474  0.009774  0.205384 -0.031023  0.157341  0.041706  0.034682  0.228407 -0.118661 -0.084567 -0.218582 -0.160545 -0.083782  0.098830  0.133829  0.096436  0.244502  0.227168  0.000758 -0.059055  0.157909  0.104076 -0.039398  0.018939  0.008092 -0.031068  0.047122 -0.011579  0.070356  0.041036 -0.093181  0.113137 -0.033564 -0.112356 0.114720 -0.104074 -0.129688 -0.0


KeyboardInterrupt



In [12]:
print(df1[df1['user_id'] == 240].sample(10))

        user_id  item_id  timestamp  \
213419      240      240  429228111   
140787      240    33103  429252495   
192940      240    18249  429246865   
140043      240    17741  429252593   
215335      240      240  429228111   
43359       240     1862  429274419   
68653       240    21332  429205987   
215512      240      240  429226576   
139876      240    26717  429252618   
1583        240      240  429284397   

                                                     text  
213419  RT @USER : Setareh Tajik was a 17 years old Af...  
140787  We 've done the maths , so you do n't have to ...  
192940  Nika Shahkarami was a bright talented 16 yro ,...  
140043  RT @USER : Thank you ! #MahsaAmini #مهسا_امینی...  
215335  RT @USER : Setareh Tajik was a 17 years old Af...  
43359   Hasti Norayei , 7 years old Shot in the head b...  
68653   RT @USER : The pitch invader during the Portug...  
215512  Zakaria Khial was shot in the head by the secu...  
139876  RT @USER : Navid Behbo

In [18]:
df_wiki = pd.read_csv('tgn/data/wikipedia.csv')

In [ ]:
from NormalizeTweets import normalizeTweet
bert_preprocess_model = hub.KerasLayer('https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3')
bert_model = hub.KerasLayer('https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-12_H-128_A-2/1')


# Gensim
# Modelli dedicati a Twitter

def getBert(text):
#    text_preprocessed = bert_preprocess_model([normalizeTweet(text)])
    text_preprocessed = bert_preprocess_model([text])
    bert_results = bert_model(text_preprocessed)
    return bert_results["pooled_output"][0, :]

In [20]:
print(df_wiki['user_id'].value_counts().index.tolist()[:10])
print(df_wiki['user_id'].value_counts().values.tolist()[:10])

[-0.06207875323834008, 3.636668725934708, 7.335416205107756, 11.034163684280806, 14.732911163453851, 18.4316586426269, 22.13040612179995, 29.527901080146044, 25.829153600972997, 36.92539603849214]
[155888, 1194, 189, 94, 36, 26, 10, 6, 6, 4]


In [21]:
print(df_wiki['item_id'].value_counts().index.tolist()[:10])
print(df_wiki['item_id'].value_counts().values.tolist()[:10])

[-0.04467350387941031, 12.863428055581613, 25.77152961504264, 38.679631174503655, 51.58773273396469, 141.9444436501919, 64.49583429342572]
[157013, 414, 33, 6, 5, 2, 1]


In [ ]:
from igraph import *#this info read in from our .csv files

g = Graph(directed=True)
g.add_vertices(list(vertices))
g.add_edges(edges)



visual_style = {}
visual_style["layout"] = g.layout("circular")
visual_style["vertex_size"] = 3
visual_style["vertex_color"] = "white"
visual_style["vertex_shape"] = "square"
visual_style["vertex_frame_color"] = "white"
visual_style["margin"] = 100
visual_style["bbox"] = (500, 500) # change to 5000, 5000 for full graph
visual_style["edge_curved"] = False
visual_style["vertex_label_size"] = 7

for e in g.es:
    src, targ = e.tuple
    if g.are_connected(targ, src):
        e["color"] = "green"           #response and response
        e["arrow_size"] = 0                       
    else:
        e["color"] = "blue"            #response
        e["arrow_size"] = 1

plot(g, **visual_style)

In [ ]:
bert_preprocess_model = hub.KerasLayer('https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3')
text_test = ['this is such an amazing movie! what the heck.']
text_preprocessed = bert_preprocess_model(text_test)

print(f'Keys       : {list(text_preprocessed.keys())}')
print(f'Shape      : {text_preprocessed["input_word_ids"].shape}')
print(f'Word Ids   : {text_preprocessed["input_word_ids"][0, :12]}')
print(f'Input Mask : {text_preprocessed["input_mask"][0, :12]}')
print(f'Type Ids   : {text_preprocessed["input_type_ids"][0, :12]}')

In [ ]:
bert_model = hub.KerasLayer('https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-12_H-128_A-2/1')
bert_results = bert_model(text_preprocessed)
print(f'Pooled Outputs Shape:{bert_results["pooled_output"].shape}')
print(f'Pooled Outputs Values:{bert_results["pooled_output"][0, :]}')
print(f'Sequence Outputs Shape:{bert_results["sequence_output"].shape}')
print(f'Sequence Outputs Values:{bert_results["sequence_output"][0, :12]}')